In [ ]:
import RPi.GPIO as io
import os
import time
import datetime
from gpiozero import Robot, DigitalInputDevice
from time import sleep

kp=1.0
ki=0.2
kd=0.0


integral=0
derivative=0

setp=18.0 # target temperature
pv=0.0 # current temperature

error=setp-pv
olderror=error
correction=0    
    
#writes the current PID to the PID.txt file for access by remote connection
#the PID number is necessary for stopping the python file from remote connection
os.chdir("/")

TimeToChange = 5*60 # IN SECONDS
with open("home/pi/Desktop/pid.txt", 'w') as PID:
   PID.write(str(os.getpid()))
#sets the pin numbering mode to GPIO number
io.setmode(io.BCM)
#sets up pin "24" as an io.in pin
io.setup(24, io.IN) # this is connected to the temperature sensor

io.setup(18, io.OUT) # connected to the L293D EN1,2 pin, which controls the heater

pwm=io.PWM(18, 100) # pulse width modulation on the 18th GPIO pin, at 100Hz

temperature=0;
alltext="";
counter=-1

#loop
try:
    while True: #infinite loop
       ###it makes a timestamp for the data that is being collected
       # appends the entry to testdata.csv
       with open("sys/bus/w1/devices/28-000008fe746d/w1_slave", 'r') as source:
           alltext=source.read()
           while alltext[counter]!="=":
               counter=counter-1;
           temperature=float(alltext[counter+1:])/1000
           source.close()
           
       with open("home/pi/Desktop/data.csv", 'a+') as data:
       #get date technique
           dateNow=str(datetime.datetime.now().strftime("%m/%d"))
           timeNow=str(int(datetime.datetime.now().strftime("%H"))*60+int(datetime.datetime.now().strftime("%M")))
       #writes comma separated date or time stamp and the ***input from the pin/file***
    
           entry = dateNow + "," + timeNow + "," + str(temperature) + "," + str(io.input(24)) + "\n"
           data.write(entry)
           data.close()
       #prints to console for easy debugging
       print(entry)
       
       ### PID SECTION
       
       pv = temperature
       error = setp - pv # positive when heating is needed
       
       integral = integral+error*TimeToChange
       derivative = (error-olderror)/TimeToChange
        
       correction=0
       correction += kp*error
       correction += ki*integral
       correction += kd*derivative
       
       if correction<0:
           correction=0; # no negative duty cycle allowed
       elif correction>100:
           correction=100 # max duty cycle
        
       olderror=error
       
       pwm.SetDutyCycle(correction) # from 0 to 100
       
       time.sleep(TimeToChange) # gap between each collection cycle
       
except KeyboardInterrupt: # so that the cleanup functions can still be run
    print("Ending program at " + str(timeNow))
    
pwm.stop()
io.cleanup()